In [4]:
import pandas as pd
import numpy as np

import traceback
import psycopg2

from datetime import datetime
from datetime import timedelta
import os
import re 

In [79]:
date = datetime.now().strftime('%d-%m-%Y')

## Funcuiones

In [71]:
def replace_status(x):
    try:
        replaced_status = df_status[df_status['Status'] == x]['HomologacionBOB'].values[0]
        return replaced_status
    except IndexError:
        return None

In [39]:
def split_names(name:str, first:bool=True, split_by=' ') -> str:
    name = name.split(split_by, maxsplit=1)
    if first:
        return name[0].replace(',','')
    return name[1].replace(',','')

In [9]:
def extractCarrier(df, carrier_name, carrier_column):
    df =  df[df[carrier_column] == carrier_name]
    return df

In [18]:
def innerJoinBOB(new_policies, bob_policies):
    upload = list(set(new_policies).intersection(set(bob_policies)) )
    return upload

In [5]:
def limpiar_df(df) -> pd.DataFrame:
    df = df.dropna(axis='columns', how='all').dropna( how='all')
    return df

In [11]:
df_b = pd.read_csv('//mnt/c/Users/cesar/Desktop/claro/bob_20230110.csv', encoding='latin-1', low_memory=False)
df_b.head(3)

,Book of Business: BOB Name,Account Executive Name,Carrier,NPN AOR,Agent AOR,Internal Agent,NPN Internal Agent,Contact,Policy Number,Source,...,Email,Metal Level,Zip Code,Book of Business: Last Modified Date,Note,Claro Type,Previous BOB ID,Carrier State line of Business,Book of Business: ID,Account Executive
0,BOB# 00000001,Miguel Bautista,Molina,1647487.0,Amarilys Perez Lazo,Amarilys Perez Lazo,1647487.0,Amarilys Perez Lazo,FLE192951620161,Direct,...,NaN,Silver,NaN,12/20/2022,NaN,Renewals,NaN,Molina Florida,a1C1G000007eLyj,
1,BOB# 00000002,Miguel Bautista,Molina,1647487.0,Amarilys Perez Lazo,Amarilys Perez Lazo,1647487.0,Amarilys Perez Lazo,FLE192921397071,Direct,...,NaN,Bronze,NaN,12/20/2022,NaN,Renewals,NaN,Molina Florida,a1C1G000007eLyk,
2,BOB# 00000003,Miguel Bautista,Molina,1647487.0,AMARILYS PEREZ LAZO,Amarilys Perez Lazo,1647487.0,Amarilys Perez Lazo,FLE192921385481,Direct,...,NaN,Bronze,NaN,12/20/2022,V-Actualización 09-01,Renewals,NaN,Molina Florida,a1C1G000007eLyl,


In [6]:
df_ameritas = limpiar_df(pd.read_excel('//mnt/c/Users/cesar/Desktop/claro/life/Ameritas/Ameritas 01-16-2023.xls'))
df_ameritas.head()

,Insured Name,Company Code,Policy Number,Writing Agent Name,Writing Agent ID,Writing Agent Profile,Servicing Agent Name,Servicing Agent ID,Servicing Agent Profile,Product Name,Issue Date,Policy Status
0,"AMADOR, JULLIANNA",100,CI60002150,"ARAQUE, RONNY",AG00105858,1,"ARAQUE, RONNY",AG00105858,1,FLX Living Benefits Index UL Guideline Premium...,04/15/2022,Active/Premium Paying
1,"DIAZ, EDWARD A",100,CI10030745,"RAMOS, WALTER",AG00122703,1,"RAMOS, WALTER",AG00122703,1,FLX Living Benefits Index UL Guideline Premium...,05/18/2021,Active/Premium Paying
2,"GONZALEZ, DANIELA F",100,CI10032287,"TAPIAS, VIRNA",AG00111178,1,"TAPIAS, VIRNA",AG00111178,1,FLX Living Benefits Index UL Guideline Premium...,06/24/2021,Surrendered
3,"MURILLO, RAFAEL A",100,CI10032145,"TAPIAS, VIRNA",AG00111178,1,"TAPIAS, VIRNA",AG00111178,1,FLX Living Benefits Index UL Guideline Premium...,06/24/2021,Active/Premium Paying
4,"MURILLO, VIRNA A",100,CI10033924,"TAPIAS, VIRNA",AG00111178,1,"TAPIAS, VIRNA",AG00111178,1,FLX Living Benefits Index UL Guideline Premium...,10/01/2021,Active/Premium Paying


## Homologación

In [7]:
df_ameritas.columns

Index(['Insured Name', 'Company Code', 'Policy Number', 'Writing Agent Name',
       'Writing Agent ID', 'Writing Agent Profile', 'Servicing Agent Name',
       'Servicing Agent ID', 'Servicing Agent Profile', 'Product Name',
       'Issue Date', 'Policy Status'],
      dtype='object')

In [8]:
df_ameritas = df_ameritas.rename(
    columns={
        'Policy Number':'PolicyNumber',
        'Insured Name':'Insured',
        'Writing Agent Name':'Agent AOR',
        'Policy Status':'Status',
        'Issue Date':'EffectiveDate',
        'Writing Agent ID':'AgentNumber',
        'Product Name':'Product'
        
        
    }
)

### Extraer las polizas de National Life

In [43]:
df_b = extractCarrier(df_b, 'Ameritas', 'Carrier')

In [44]:
upload = innerJoinBOB( df_b['Policy Number'].values, df_ameritas['PolicyNumber'].values)

In [45]:
upload

['CI10030745',
 'CI60006169',
 'CI10032145',
 'CI60006528',
 'CI10032287',
 'CI60002150',
 'CI10031759',
 'CI10035123',
 'CI10033924']

In [46]:
df_b.columns

Index(['Book of Business: BOB Name', 'Account Executive Name', 'Carrier',
       'NPN AOR', 'Agent AOR', 'Internal Agent', 'NPN Internal Agent',
       'Contact', 'Policy Number', 'Source', 'Referrals Id', 'Status',
       'Effective Date', 'Policy Term Date', 'Member First Name',
       'Member Last Name', 'DOB', 'State', 'Members', 'Policy Type',
       'Automatic Renewal', 'Line of Business',
       'Last Status Modification Date', 'Loading Date',
       'Book of Business: Created Date', 'Book of Business: Record Type',
       'Agency', 'Plan', 'Final Premium Currency', 'Final Premium',
       'Subsidy Currency', 'Subsidy', 'Annual Premium Currency',
       'Annual Premium', 'Renewal Type', 'AEP 2023', 'Renewal 2023',
       'All 2023', 'Account Name', 'Paid Thru Date', 'Days Past Due', 'County',
       'City', 'Phone Number', 'Email', 'Metal Level', 'Zip Code',
       'Book of Business: Last Modified Date', 'Note', 'Claro Type',
       'Previous BOB ID', 'Carrier State line of Busi

In [47]:
df_act = df_b[df_b['Policy Number'].isin(upload)][['Book of Business: ID','Policy Number']]

In [48]:
df_upload = df_ameritas[df_ameritas['PolicyNumber'].isin(upload)]

In [49]:
df_upload = df_upload.merge(
    df_act,
    left_on='PolicyNumber',
    right_on='Policy Number'
)

In [50]:
df_template = pd.read_csv('//mnt/c/Users/cesar/Desktop/claro/Format_Create_BOB.csv').head(0)
df_template

,Carrier___,LOB____,Carrier,Agent AOR,NPN AOR,Contact____,NPN_Internal____,Contact,Policy Number,Source,...,Subsidy,Annual Premium,Paid Thru Date,Days Past Due,Phone Number,Email,Metal Level,Zip Code,Claro Type,Carrier State line of Business


In [53]:
df_upload['Member First Name'] = df_upload['Insured'].apply(lambda x : split_names(x, first=False))
df_upload['Member Last Name'] = df_upload['Insured'].apply(lambda x : split_names(x))

In [56]:
df_upload = df_upload.drop(columns=['Insured'])

In [58]:
df_upload.head(3)

,Company Code,PolicyNumber,Agent AOR,AgentNumber,Writing Agent Profile,Servicing Agent Name,Servicing Agent ID,Servicing Agent Profile,Product,EffectiveDate,Status,Book of Business: ID,Policy Number,Member First Name,Member Last Name
0,100,CI60002150,"ARAQUE, RONNY",AG00105858,1,"ARAQUE, RONNY",AG00105858,1,FLX Living Benefits Index UL Guideline Premium...,04/15/2022,Active/Premium Paying,a1C1G000008dkXy,CI60002150,JULLIANNA,AMADOR
1,100,CI10030745,"RAMOS, WALTER",AG00122703,1,"RAMOS, WALTER",AG00122703,1,FLX Living Benefits Index UL Guideline Premium...,05/18/2021,Active/Premium Paying,a1C1G000008gdBw,CI10030745,EDWARD A,DIAZ
2,100,CI10032287,"TAPIAS, VIRNA",AG00111178,1,"TAPIAS, VIRNA",AG00111178,1,FLX Living Benefits Index UL Guideline Premium...,06/24/2021,Surrendered,a1C1G000008gdC1,CI10032287,DANIELA F,GONZALEZ


In [69]:
df_status = pd.read_excel('/mnt/c/Users/cesar/Desktop/claro/homologacion/homologacionpolicies.xlsx')
df_status.head(3)

,Carrier Name,Status,HomologacionBOB
0,National Life,Active,Active
1,National Life,No Active,No Active
2,National Life,Issued,Issued


In [70]:
df_status['HomologacionBOB']

0                   Active
1                No Active
2                   Issued
3                No Active
4           Pending Lapsed
5                   Issued
6                   CLOSED
7                   CLOSED
8                   Issued
9                   Issued
10                 PENDING
11    Upcoming Anniversary
12        Request Complete
13    Upcoming Anniversary
14                    Null
15                 PENDING
16                 PENDING
17              Incomplete
18                  Active
19             Surrendered
20                  Active
21                  CLOSED
Name: HomologacionBOB, dtype: object

In [73]:
df_upload['Status'] = df_upload['Status'].apply(lambda x : replace_status(x) )

In [76]:
df_upload.columns

Index(['Company Code', 'PolicyNumber', 'Agent AOR', 'AgentNumber',
       'Writing Agent Profile', 'Servicing Agent Name', 'Servicing Agent ID',
       'Servicing Agent Profile', 'Product', 'EffectiveDate', 'Status',
       'Book of Business: ID', 'Policy Number', 'Member First Name',
       'Member Last Name'],
      dtype='object')

In [83]:
df_upload = df_upload[['PolicyNumber', 'Agent AOR', 'AgentNumber', 'Product', 'Status', 'Book of Business: ID', 'Member First Name','Member Last Name', 'Servicing Agent ID']]

In [84]:
df_upload

,PolicyNumber,Agent AOR,AgentNumber,Product,Status,Book of Business: ID,Member First Name,Member Last Name,Servicing Agent ID
0,CI60002150,"ARAQUE, RONNY",AG00105858,FLX Living Benefits Index UL Guideline Premium...,Active,a1C1G000008dkXy,JULLIANNA,AMADOR,AG00105858
1,CI10030745,"RAMOS, WALTER",AG00122703,FLX Living Benefits Index UL Guideline Premium...,Active,a1C1G000008gdBw,EDWARD A,DIAZ,AG00122703
2,CI10032287,"TAPIAS, VIRNA",AG00111178,FLX Living Benefits Index UL Guideline Premium...,Surrendered,a1C1G000008gdC1,DANIELA F,GONZALEZ,AG00111178
3,CI10032145,"TAPIAS, VIRNA",AG00111178,FLX Living Benefits Index UL Guideline Premium...,Active,a1C1G000008gdC6,RAFAEL A,MURILLO,AG00111178
4,CI10033924,"TAPIAS, VIRNA",AG00111178,FLX Living Benefits Index UL Guideline Premium...,Active,a1C1G000008gdCB,VIRNA A,MURILLO,AG00111178
5,CI10031759,"RAMOS, WALTER",AG00122703,FLX Living Benefits Index UL Guideline Premium...,Active,a1C1G000008gdCG,DANIEL S,RAMOS,AG00122703
6,CI60006169,"CASTANO PADILLA, FATIMA",AG00132082,FLX Living Benefits Index UL Guideline Premium...,Active,a1C1G000009UG8B,JULIO,SANTOS,AG00132082
7,CI10035123,"TAPIAS, VIRNA",AG00111178,FLX Living Benefits Index UL Guideline Premium...,Active,a1C1G000008gowa,LECUNA JENNIFER C,SAYAGO,AG00111178
8,CI60006528,"CASTANO PADILLA, FATIMA",AG00132082,FLX Living Benefits Index UL Guideline Premium...,CLOSED,a1C1G000009UG7w,MEDINA JORGE R,TORRES,AG00132082


In [90]:
df_upload.rename(columns={'Book of Business: ID':'ID'}, inplace=True)

/home/cesarppz/.local/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [93]:
df_upload

,PolicyNumber,Agent AOR,AgentNumber,Product,Status,ID,Member First Name,Member Last Name,Servicing Agent ID
0,CI60002150,"ARAQUE, RONNY",AG00105858,FLX Living Benefits Index UL Guideline Premium...,Active,a1C1G000008dkXy,JULLIANNA,AMADOR,AG00105858
1,CI10030745,"RAMOS, WALTER",AG00122703,FLX Living Benefits Index UL Guideline Premium...,Active,a1C1G000008gdBw,EDWARD A,DIAZ,AG00122703
2,CI10032287,"TAPIAS, VIRNA",AG00111178,FLX Living Benefits Index UL Guideline Premium...,Surrendered,a1C1G000008gdC1,DANIELA F,GONZALEZ,AG00111178
3,CI10032145,"TAPIAS, VIRNA",AG00111178,FLX Living Benefits Index UL Guideline Premium...,Active,a1C1G000008gdC6,RAFAEL A,MURILLO,AG00111178
4,CI10033924,"TAPIAS, VIRNA",AG00111178,FLX Living Benefits Index UL Guideline Premium...,Active,a1C1G000008gdCB,VIRNA A,MURILLO,AG00111178
5,CI10031759,"RAMOS, WALTER",AG00122703,FLX Living Benefits Index UL Guideline Premium...,Active,a1C1G000008gdCG,DANIEL S,RAMOS,AG00122703
6,CI60006169,"CASTANO PADILLA, FATIMA",AG00132082,FLX Living Benefits Index UL Guideline Premium...,Active,a1C1G000009UG8B,JULIO,SANTOS,AG00132082
7,CI10035123,"TAPIAS, VIRNA",AG00111178,FLX Living Benefits Index UL Guideline Premium...,Active,a1C1G000008gowa,LECUNA JENNIFER C,SAYAGO,AG00111178
8,CI60006528,"CASTANO PADILLA, FATIMA",AG00132082,FLX Living Benefits Index UL Guideline Premium...,CLOSED,a1C1G000009UG7w,MEDINA JORGE R,TORRES,AG00132082


In [94]:
(df_upload
.to_csv(f'/mnt/c/Users/cesar/Desktop/claro/imports/actualizacion_ameritas_{date}_.csv', index=False))